In [2]:
import os
import json
import requests
import pandas as pd
from config import api_key


In [3]:
editors_ls = ['Tuna Bar', 
              'Zahav', 
              'Pho Ga Thanh Thanh',
              'Ralphs Italian Restaurant',
              'Han Dynasty',
              'La Mula Terca',
              'Village Whiskey',
              'Grill N Dutchy',
              'Parc',
              'Philly Style Bagels'
              ]

In [4]:
SEARCH = 'https://api.yelp.com/v3/businesses/search'
API_KEY = api_key
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

def search_restaurant(str):
    """Uses Yelp Fusion API to search for a specific restaurant
    Args:
        str (str): restaurant name
    Returns:
        dict: dictionary of basic descriptions of a specified restaurant
    """

    parameters = {
        'term' : str,
        'location' : 'Philadelphia',
        'limit' : 1
    }

    data_json = requests.get(url = SEARCH, params=parameters, headers = HEADERS).json()
    
    return data_json['businesses']


def search_multiple(ls):
    """Search for multiple restaurants in a list using Yelp API
    Args:
        ls (list): list of restaurants
    Returns:
        restaurant_ls (ls): list of dictionaries for each restaurant containing basic descriptions
    """
    restaurant_ls = []
    for restaurant in ls:
        restaurant_ls.extend(search_restaurant(restaurant))        
    return restaurant_ls

def get_ids(ls):
    """gets restaurant ids to be used for different endpoints
    Args:
        ls (list): list of restaurants
    Returns:
        id_ls (list): list of restaurant ids
    """
    id_ls = []
    [id_ls.append(r['id']) for r in search_multiple(ls)]
        
    return id_ls

def get_details(ls):
    details_ls = []
    for id in ls:
        url_details = 'https://api.yelp.com/v3/businesses/{}'.format(id)
        details_ls.append(requests.get(url = url_details, headers = HEADERS).json())
    
    return details_ls

def get_reviews(ls):
    reviews_ls = []
    for id in ls:
        url_reviews = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(id)
        reviews_ls.append(requests.get(url = url_reviews, headers = HEADERS).json())
    
    return reviews_ls
     
    

In [5]:
# list of business IDs for restaurants in "editor's list"
id_ls = get_ids(editors_ls)
id_ls

['MUTTqe8uqyMdBl186RmNeA',
 'ctHjyadbDQAtUFfkcAFEHw',
 'e3fQkykBr9V9jr1g9VWliw',
 'j99PtxlKTW_u5alE4jiqHQ',
 'qISf5ojuYbD9h71NumGUQA',
 'b1okJ5qEPCIHbIgQ1NGyXQ',
 'EtKSTHV5Qx_Q7Aur9o4kQQ',
 'l2H2TDEpnAGSi9SG6evqww',
 'j-qtdD55OLfSqfsWuQTDJg',
 '9kYR383I5IumdKEFTAsrxQ']

In [6]:
# contains details about each restaurant in list including price scale and photos
details_ls = get_details(id_ls)

In [7]:
details_ls

[{'id': 'MUTTqe8uqyMdBl186RmNeA',
  'alias': 'tuna-bar-philadelphia',
  'name': 'Tuna Bar',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/zWwgcC178Z0LI09H1Os4jg/o.jpg',
  'is_claimed': True,
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/tuna-bar-philadelphia?adjust_creative=_U1xfxPia8AwCYodf0zevg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=_U1xfxPia8AwCYodf0zevg',
  'phone': '+12152388862',
  'display_phone': '(215) 238-8862',
  'review_count': 275,
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'sushi', 'title': 'Sushi Bars'}],
  'rating': 4.0,
  'location': {'address1': '205 Race St',
   'address2': None,
   'address3': '',
   'city': 'Philadelphia',
   'zip_code': '19106',
   'country': 'US',
   'state': 'PA',
   'display_address': ['205 Race St', 'Philadelphia, PA 19106'],
   'cross_streets': ''},
  'coordinates': {'latitude': 39.953949, 'longitude': -75.1432262},
  'photos': ['https://s3-media4.fl.yelpcdn.co

In [8]:
# clean up json and drop unnecessary keys
for x in details_ls:
    keys = ['alias','coordinates', 'hours', 'is_claimed', 'is_closed','transactions','review_count', 'phone','messaging']
    for key in keys:
        x.pop(key, None)

In [9]:
details_ls

[{'id': 'MUTTqe8uqyMdBl186RmNeA',
  'name': 'Tuna Bar',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/zWwgcC178Z0LI09H1Os4jg/o.jpg',
  'url': 'https://www.yelp.com/biz/tuna-bar-philadelphia?adjust_creative=_U1xfxPia8AwCYodf0zevg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=_U1xfxPia8AwCYodf0zevg',
  'display_phone': '(215) 238-8862',
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'sushi', 'title': 'Sushi Bars'}],
  'rating': 4.0,
  'location': {'address1': '205 Race St',
   'address2': None,
   'address3': '',
   'city': 'Philadelphia',
   'zip_code': '19106',
   'country': 'US',
   'state': 'PA',
   'display_address': ['205 Race St', 'Philadelphia, PA 19106'],
   'cross_streets': ''},
  'photos': ['https://s3-media4.fl.yelpcdn.com/bphoto/zWwgcC178Z0LI09H1Os4jg/o.jpg',
   'https://s3-media4.fl.yelpcdn.com/bphoto/k0Oho5qzxrBqa5qQdsjwJA/o.jpg',
   'https://s3-media3.fl.yelpcdn.com/bphoto/EJds-DChsPTk7b_ZRkJiPQ/o.jpg'],
  'pric

In [10]:
# convert list of dictionaries to df and make a copy
details_df = pd.DataFrame(details_ls, index=None)
# details = details.copy()
# details_df.to_csv('restaurant_details.csv')

In [11]:
details_df.columns

Index(['id', 'name', 'image_url', 'url', 'display_phone', 'categories',
       'rating', 'location', 'photos', 'price'],
      dtype='object')

In [12]:
details_df = details_df.rename(columns= {'id':'RestaurantID', 'name':'Name', 'image_url': 'Display_photo', 'display_phone':'Phone', 'categories':'Category', 'rating':'Rating', 'location':'Address','photos':'Photos',"price":"Price"} )


In [13]:
Restaurants_df = details_df[['RestaurantID', 'Name', 'Address', 'Phone', 'Rating', 'Category', 'Price', 'url']]

In [14]:
Restaurants_df

,RestaurantID,Name,Address,Phone,Rating,Category,Price,url
0,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,"{'address1': '205 Race St', 'address2': None, ...",(215) 238-8862,4.0,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",$$,https://www.yelp.com/biz/tuna-bar-philadelphia...
1,ctHjyadbDQAtUFfkcAFEHw,Zahav,"{'address1': '237 St James Pl', 'address2': ''...",(215) 625-8800,4.5,"[{'alias': 'mideastern', 'title': 'Middle East...",$$$$,https://www.yelp.com/biz/zahav-philadelphia?ad...
2,e3fQkykBr9V9jr1g9VWliw,Pho Ga Thanh Thanh,"{'address1': '1100 Washington Ave', 'address2'...",(215) 427-0483,4.5,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",$$,https://www.yelp.com/biz/pho-ga-thanh-thanh-ph...
3,j99PtxlKTW_u5alE4jiqHQ,Ralphs Italian Restaurant,"{'address1': '760 S 9th St', 'address2': '', '...",(215) 627-6011,3.5,"[{'alias': 'italian', 'title': 'Italian'}]",$$,https://www.yelp.com/biz/ralphs-italian-restau...
4,qISf5ojuYbD9h71NumGUQA,Han Dynasty,"{'address1': '123 Chestnut St', 'address2': ''...",(215) 922-1888,4.0,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",$$,https://www.yelp.com/biz/han-dynasty-philadelp...
5,b1okJ5qEPCIHbIgQ1NGyXQ,La Mula Terca,"{'address1': '2053 S Beechwood St', 'address2'...",(267) 761-5194,4.5,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",$$,https://www.yelp.com/biz/la-mula-terca-philade...
6,EtKSTHV5Qx_Q7Aur9o4kQQ,Village Whiskey,"{'address1': '118 S 20th St', 'address2': '', ...",(215) 665-1088,4.0,"[{'alias': 'newamerican', 'title': 'American (...",$$,https://www.yelp.com/biz/village-whiskey-phila...
7,l2H2TDEpnAGSi9SG6evqww,Grill N Dutchy,"{'address1': '5522 Germantown Ave', 'address2'...",(215) 849-3500,4.5,"[{'alias': 'caribbean', 'title': 'Caribbean'}]",NaN,https://www.yelp.com/biz/grill-n-dutchy-philad...
8,j-qtdD55OLfSqfsWuQTDJg,Parc,"{'address1': '227 S 18th St', 'address2': '', ...",(215) 545-2262,4.0,"[{'alias': 'french', 'title': 'French'}, {'ali...",$$$,https://www.yelp.com/biz/parc-philadelphia-2?a...
9,9kYR383I5IumdKEFTAsrxQ,Philly Style Bagels,"{'address1': '1451 E Columbia Ave', 'address2'...",,4.0,"[{'alias': 'bagels', 'title': 'Bagels'}]",$$,https://www.yelp.com/biz/philly-style-bagels-p...


In [ ]:
Restaurants_df.to_csv('archived/restaurant_details.csv')


In [15]:
Photos_df = details_df[['RestaurantID', 'Name', 'Display_photo', 'Photos']]
Photos_df

,RestaurantID,Name,Display_photo,Photos
0,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,https://s3-media4.fl.yelpcdn.com/bphoto/zWwgcC...,[https://s3-media4.fl.yelpcdn.com/bphoto/zWwgc...
1,ctHjyadbDQAtUFfkcAFEHw,Zahav,https://s3-media1.fl.yelpcdn.com/bphoto/H52tmZ...,[https://s3-media1.fl.yelpcdn.com/bphoto/H52tm...
2,e3fQkykBr9V9jr1g9VWliw,Pho Ga Thanh Thanh,https://s3-media3.fl.yelpcdn.com/bphoto/o70MEO...,[https://s3-media3.fl.yelpcdn.com/bphoto/o70ME...
3,j99PtxlKTW_u5alE4jiqHQ,Ralphs Italian Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/EOmzNs...,[https://s3-media4.fl.yelpcdn.com/bphoto/EOmzN...
4,qISf5ojuYbD9h71NumGUQA,Han Dynasty,https://s3-media3.fl.yelpcdn.com/bphoto/fFU7ca...,[https://s3-media3.fl.yelpcdn.com/bphoto/fFU7c...
5,b1okJ5qEPCIHbIgQ1NGyXQ,La Mula Terca,https://s3-media1.fl.yelpcdn.com/bphoto/Z2OXeH...,[https://s3-media1.fl.yelpcdn.com/bphoto/Z2OXe...
6,EtKSTHV5Qx_Q7Aur9o4kQQ,Village Whiskey,https://s3-media2.fl.yelpcdn.com/bphoto/_hJndw...,[https://s3-media2.fl.yelpcdn.com/bphoto/_hJnd...
7,l2H2TDEpnAGSi9SG6evqww,Grill N Dutchy,https://s3-media1.fl.yelpcdn.com/bphoto/tdzztl...,[https://s3-media1.fl.yelpcdn.com/bphoto/tdzzt...
8,j-qtdD55OLfSqfsWuQTDJg,Parc,https://s3-media3.fl.yelpcdn.com/bphoto/0vGzdK...,[https://s3-media3.fl.yelpcdn.com/bphoto/0vGzd...
9,9kYR383I5IumdKEFTAsrxQ,Philly Style Bagels,https://s3-media4.fl.yelpcdn.com/bphoto/XZQTUb...,[https://s3-media4.fl.yelpcdn.com/bphoto/XZQTU...


In [16]:
reviews_ls = get_reviews(id_ls)

In [17]:
reviews_ls

[{'reviews': [{'id': 'tF0v3ttaMZOxcvQIyBb2kg',
    'url': 'https://www.yelp.com/biz/tuna-bar-philadelphia?adjust_creative=_U1xfxPia8AwCYodf0zevg&hrid=tF0v3ttaMZOxcvQIyBb2kg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=_U1xfxPia8AwCYodf0zevg',
    'text': 'Before heading to the Mann Center for a concert, I found this place on Yelp. \nThe pics alone had me bookmark this Japanese spot.\n\nHere is what we had:\n-...',
    'rating': 5,
    'time_created': '2022-09-27 08:49:23',
    'user': {'id': 'w0Gp4qYFLhB6PbKrPhn6Tw',
     'profile_url': 'https://www.yelp.com/user_details?userid=w0Gp4qYFLhB6PbKrPhn6Tw',
     'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/aAEdRIgEkMIptE7fwXLA2Q/o.jpg',
     'name': 'Pamela L.'}},
   {'id': 'mOXHImYYYCmndsgtmoBKLw',
    'url': 'https://www.yelp.com/biz/tuna-bar-philadelphia?adjust_creative=_U1xfxPia8AwCYodf0zevg&hrid=mOXHImYYYCmndsgtmoBKLw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=_U1xfxPia8AwCY

In [19]:
reviews_df = pd.json_normalize(reviews_ls, 
                          record_path='reviews')

In [35]:
# reviews_df.columns
reviews_df = reviews_df[['id','text','rating','user.name']].dropna()

In [37]:
reviews_df.to_csv('reviews.csv')